# Trabalho Prático 3
**Grupo 22**

Alexis Correia - A102495 <br>
João Fonseca - A102512 <br>

## Problema 1

O algoritmo estendido de Euclides (EXA) aceita dois inteiros constantes  $\,a,b>0\,$  e devolve inteiros $r,s,t\,$ tais que  $\,a*s + b*t = r\,$  e  $\,r = \gcd(a,b)\,$. 

> Para além das variáveis $\,r,s,t\,$ o código requer 3 variáveis adicionais $\,r',s',t'\,$ que representam os valores de $\,r,s,t\,$ no “próximo estado”.

```Python
{INPUT  a, b}
{assume  a > 0 and b > 0}

0: r, r', s, s', t, t' = a, b, 1, 0, 0, 1
1: while r' != 0:
2:   q = r div r'
3:   r, r', s, s', t, t' = r', r − q × r', s', s − q × s', t', t − q × t' 

{OUTPUT r, s, t}
```

1. Construa um SFOTS usando BitVector’s de tamanho $n$ que descreva o comportamento deste programa.  Considere estado de erro quando $\,r=0\,$ ou alguma das variáveis atinge o “overflow”.
2. Prove, usando a metodologia dos invariantes e interpolantes, que o modelo nunca atinge o estado de erro.

## Resolução

Um SFOTS é definido  por $\:\:\Sigma\;\equiv\;\langle\,\mathsf{X}\,,\,\mathsf{next}\,,\,\mathsf{I}\,,\,\mathsf{T}\,,\,\mathsf{E}\,\rangle\:\:$. Similiar a um FOTS, os estados são constituídos pelas variáveis do programa mais o __program counter__ (pc) e, tanto o estado inicial quanto as relações de trnsição, são caracterizados por predicados. A maior diferença se encontra na existência do estado de erro.

Primeiramente, faremos algumas alterações no código. Desta maneira:
```Python
{INPUT  a, b}
{assume  a > 0 and b > 0}

0: r, r', s, s', t, t' = a, b, 1, 0, 0, 1
1: while r' != 0:
2:   q = r div r'
3:   if overflow or r=0:
4:      raise Error
5:   r, r', s, s', t, t' = r', r − q × r', s', s − q × s', t', t − q × t' 
6: stop

{OUTPUT r, s, t}
```
Agora podemos prosseguir.
- As variáveis do programa são: $ r $, $ r' $, $ s $, $ s' $, $ t $, $ t' $, $ q $ e não podemos nos esquecer do $ a $ e do $ b $
- O estado inicial é caracterizado pelo predicado: $\mathit{pc} = 0 \wedge a \ge 0 \wedge b \ge 0$
- As transições possíveis são caracterizadas das seguintes formas:
$$
\begin{array}{c}
(\mathit{pc}=0\wedge a\ge 0\wedge b\ge 0\wedge\mathit{PC}=1\wedge A=a\wedge B=b\wedge R=a\wedge R'=b\wedge S=1\wedge S'=0\wedge T=0\wedge T'=1\wedge Q=q)
\\\vee\\
(\mathit{pc}=1\wedge r'=0\wedge\mathit{PC}=6\wedge A=a\wedge B=b\wedge R=r\wedge R'=r'\wedge S=s\wedge S'=s'\wedge T=t\wedge T'=t'\wedge Q=q)
\\\vee\\
(\mathit{pc}=1\wedge r'\neq 0\wedge\mathit{PC}=2\wedge A=a\wedge B=b\wedge R=r\wedge R'=r'\wedge S=s\wedge S'=s'\wedge T=t\wedge T'=t'\wedge Q=q)
\\\vee\\
(\mathit{pc}=2\wedge\mathit{PC}=3\wedge A=a\wedge B=b\wedge R=r\wedge R'=r'\wedge S=s\wedge S'=s'\wedge T=t\wedge T'=t'\wedge Q=r\;\;\text{div}\;\; r')
\\\vee\\
(\mathit{pc}=3\wedge\;\;\text{overflow or R}\;\;\wedge\mathit{PC}=4\wedge A=a\wedge B=b\wedge R=r\wedge R'=r'\wedge S=s\wedge S'=s'\wedge T=t\wedge T'=t'\wedge Q=q)
\\\vee\\
(\mathit{pc}=3\wedge\;\;\text{Not(overflow or R)}\;\;\wedge\mathit{PC}=5\wedge A=a\wedge B=b\wedge R=r\wedge R'=r'\wedge S=s\wedge S'=s'\wedge T=t\wedge T'=t'\wedge Q=q)
\\\vee\\
(\mathit{pc}=5\wedge\mathit{PC}=1\wedge A=a\wedge B=b\wedge R=r'\wedge R'=r-q\times r'\wedge S=s'\wedge S'=s-q\times s'\wedge T=t'\wedge T'=t-q\times t'\wedge Q=q)
\\\vee\\
(\mathit{pc}=6\wedge\mathit{PC}=6\wedge A=a\wedge B=b\wedge R=r\wedge R'=r'\wedge S=s\wedge S'=s'\wedge T=t\wedge T'=t'\wedge Q=q)
\end{array}
$$
- O estado de erro acontece caso haja um ``overflow`` (ou seja, o valor de $ q $ ultrapassa um certo limite) ou quando $ r=0 $
    + Quanto ao limite do valor da variável $ q $, veremos isso mais afrente.

> Repare que: nas transições, dado que as variáveis $ r' $, $ s' $ e $ t' $ são variáveis do programa, representamos as variáveis do próximo estado por letras maiúsculas.

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import BVType, INT

n = 32 # num bits = 4 bytes

# SFOTS #
def genState(vars, s, i):
    state = {}
    for v in vars:
        if v=='pc':
            state[v] = Symbol(v+'!'+s+str(i), INT)
        else:    
            state[v] = Symbol(v+'!'+s+str(i), BVType(n))
    return state

def init(state):
    A = BVSGE(state['a'], BV(0,n)) 
    B = BVSGE(state['b'], BV(0,n))
    C = Equals(state['pc'], Int(0))
    return And(A,B,C)

def trans(curr, prox): 
    t01 = And(Equals(curr['pc'],Int(0)), BVSGE(curr['a'],BV(0,n)), BVSGE(curr['b'],BV(0,n)), Equals(prox['pc'],Int(1)),
              Equals(prox['a'],curr['a']), Equals(prox['b'],curr['b']), Equals(prox['r'],curr['a']),
              Equals(prox["r'"],curr['b']), Equals(prox['s'], BV(1,n)), Equals(prox["s'"], BV(0,n)),
              Equals(prox['t'], BV(0,n)), Equals(prox["t'"], BV(1,n)), Equals(prox['q'], curr['q']))
    
    t16 = And(Equals(curr['pc'],Int(1)), Equals(curr["r'"], BV(0,n)), Equals(prox['pc'],Int(6)), Equals(prox['a'],curr['a']),
              Equals(prox['b'],curr['b']), Equals(prox['r'],curr['r']), Equals(prox["r'"],curr["r'"]),
              Equals(prox['s'],curr['s']), Equals(prox["s'"],curr["s'"]), Equals(prox['t'],curr['t']), 
              Equals(prox['t'], curr["t'"]), Equals(prox['q'],curr['q']))
    
    t12 = And(Equals(curr['pc'],Int(0)), Not(Equals(curr["r'"], BV(0,n))), Equals(prox['pc'],Int(2)), Equals(prox['a'],curr['a']),
              Equals(prox['b'],curr['b']), Equals(prox['r'],curr['r']), Equals(prox["r'"],curr["r'"]),
              Equals(prox['s'],curr['s']), Equals(prox["s'"],curr["s'"]), Equals(prox['t'],curr['t']), 
              Equals(prox['t'], curr["t'"]), Equals(prox['q'],curr['q']))
    
    t23 = And(Equals(curr['pc'], Int(2)), Equals(prox['pc'], Int(3)), Equals(prox['a'],curr['a']), Equals(prox['b'],curr['b']),
              Equals(prox['r'],curr['r']), Equals(prox['s'],curr['s']), Equals(prox['t'],curr['t']), Equals(prox["r'"],curr["r'"]),
              Equals(prox["s'"],curr["s'"]), Equals(prox["t'"],curr["t'"]), Equals(prox['q'], BVSDiv(curr['r'],curr["r'"])))

    of_R = BVSGT(curr['q'], BVSDiv(BV(2**n -1, n), curr["r'"]))
    of_S = BVSGT(curr['q'], BVSDiv(BV(2**n -1, n), curr["s'"]))
    of_T = BVSGT(curr['q'], BVSDiv(BV(2**n -1, n), curr["t'"]))
    overflow = Or(of_R, of_S, of_T)
    R = Equals(curr['r'], BV(0, n))

    t34 = And(Equals(curr['pc'],Int(3)), Or(overflow, R), Equals(prox['pc'],Int(4)), Equals(prox['a'],curr['a']),
              Equals(prox['b'],curr['b']), Equals(prox['r'],curr['r']), Equals(prox["r'"],curr["r'"]),
              Equals(prox['s'],curr['s']), Equals(prox["s'"],curr["s'"]), Equals(prox['t'],curr['t']), 
              Equals(prox['t'], curr["t'"]), Equals(prox['q'],curr['q']))

    t35 = And(Equals(curr['pc'],Int(3)), Not(Or(overflow, R)), Equals(prox['pc'],Int(5)), Equals(prox['a'],curr['a']),
              Equals(prox['b'],curr['b']), Equals(prox['r'],curr['r']), Equals(prox["r'"],curr["r'"]),
              Equals(prox['s'],curr['s']), Equals(prox["s'"],curr["s'"]), Equals(prox['t'],curr['t']), 
              Equals(prox['t'], curr["t'"]), Equals(prox['q'],curr['q']))
    
    t51 = And(Equals(curr['pc'], Int(5)), Equals(prox['pc'], Int(1)), Equals(prox['a'],curr['a']), Equals(prox['b'],curr['b']),
              Equals(prox['r'],curr["r'"]), Equals(prox["r'"], BVSub(curr['r'], BVMul(curr['q'],curr["r'"]))),
              Equals(prox['s'],curr["s'"]), Equals(prox["s'"], BVSub(curr['s'], BVMul(curr['q'],curr["s'"]))),
              Equals(prox['t'],curr["t'"]), Equals(prox["t'"], BVSub(curr['t'], BVMul(curr['q'],curr["t'"]))),
              Equals(prox['q'], curr['q']))

    t66 = And(Equals(curr['pc'], Int(6)), Equals(prox['pc'], Int(6)), Equals(prox['a'],curr['a']), Equals(prox['b'],curr['b']),
              Equals(prox['r'],curr['r']), Equals(prox['s'],curr['s']), Equals(prox['t'],curr['t']), Equals(prox["r'"],curr["r'"]),
              Equals(prox["s'"],curr["s'"]), Equals(prox["t'"],curr["t'"]), Equals(prox['q'],curr['q']))

    return Or(t01, t16, t12, t23, t34, t35, t51, t66)

def error(state):
    return Equals(state['pc'],Int(4))

Como podemos ver no código acima, o limite da variável $ q $ depende dos valores das outras variáveis (nomeadamente: $r'$, $s'$ e $t'$). Isso se deve ao facto de que, mais adiante no código, é feito diversas multiplicações entre $q$ e essas outras variáveis mencionadas.

Com as funções acima prontas, podemos escrever a função ```genTrace```. Dessa forma, podemos utilizar o solver ```z3``` e escrever os traços.

In [2]:
def genTrace(vars,init,trans,error,n): 
    with Solver(name="z3") as s:
        X = [genState(vars,'X',i) for i in range(n+1)]
        I = init(X[0])
        Tks = [ trans(X[i],X[i+1]) for i in range(n) ]
        E = [error(X[i]) for i in range(n)]
        if s.solve([I,And(Tks),Not(And(E))]):
            for i in range(n):
                print("Estado:",i)
                for v in X[i]:
                    print("          ",v,'=',s.get_value(X[i][v])) ###
        else:
            print("ERRO")

var = ['pc', 'a', 'b', 'r', "r'", 's', "s'", 't', "t'", 'q']

genTrace(var, init, trans, error, 20) ##

Estado: 0
           pc = 0
           a = 1050881_32
           b = 32768_32
           r = 2140049280_32
           r' = 4290803776_32
           s = 3178689246_32
           s' = 27525120_32
           t = 1766055920_32
           t' = 1766055920_32
           q = 2147748354_32
Estado: 1
           pc = 2
           a = 1050881_32
           b = 32768_32
           r = 2140049280_32
           r' = 4290803776_32
           s = 3178689246_32
           s' = 27525120_32
           t = 1766055920_32
           t' = 1766055920_32
           q = 2147748354_32
Estado: 2
           pc = 3
           a = 1050881_32
           b = 32768_32
           r = 2140049280_32
           r' = 4290803776_32
           s = 3178689246_32
           s' = 27525120_32
           t = 1766055920_32
           t' = 1766055920_32
           q = 4294966782_32
Estado: 3
           pc = 5
           a = 1050881_32
           b = 32768_32
           r = 2140049280_32
           r' = 4290803776_32
           s = 31

Agora, ntes de seguirmos para a metodologia dos invariantes e interpolantes, precisamos definir mais algumas funções. Nomeadamente, a função ```invert```, que codifica a transição inversa, e outras funções auxiliares.

In [3]:
def invert(trans): # genTrace: <var, error, invert(trans), init, n>
    return lambda curr, nxt: trans(nxt, curr)

In [4]:
import itertools

def baseName(s):
    return ''.join(list(itertools.takewhile(lambda x: x!='!', s)))

def rename(form,state):
    vs = get_free_variables(form)
    pairs = [ (x,state[baseName(x.symbol_name())]) for x in vs ]
    return form.substitute(dict(pairs))

def same(state1,state2):
    return And([Equals(state1[x],state2[x]) for x in state1])

Com tudo pronto, podemos prosseguir e testar se este é, de facto, seguro. Para isso,  utilizaremos a metodologia dos invariantes e interpolantes e demonstraremos que o modelo nunca atinge o estado de erro.

In [5]:
def model_checking(vars,init,trans,error,N,M):
    with Solver(name="z3") as solver:
        
        # Criar todos os estados que poderão vir a ser necessários.
        X = [genState(vars,'X',i) for i in range(N+1)]
        Y = [genState(vars,'Y',i) for i in range(M+1)]
        transt = invert(trans)
        
        # Estabelecer a ordem pela qual os pares (n,m) vão surgir. Por exemplo:
        order = sorted([(a,b) for a in range(1,N+1) for b in range(1,M+1)],key=lambda tup:tup[0]+tup[1]) 
        
        # Step 1 implícito na ordem de 'order' e nas definições de Rn, Um.
        for (n,m) in order:      
            # Step 2. 
            I = init(X[0])
            Tn = And([trans(X[i], X[i+1]) for i in range(n)])
            Rn = And(I, Tn)
            
            E = error(Y[0])
            Bm = And([transt(Y[i], Y[i+1]) for i in range(m)])
            Um = And(E, Bm)
            
            Vnm = And(Rn, same(X[n], Y[m]), Um)
            if solver.solve([Vnm]):
                print("> O sistema é inseguro.")
                return
            else:
                # Step 3. 
                A = And(Rn, same(X[n], Y[m]))
                B = Um
                C = binary_interpolant(A, B, solver_name='msat')
                
                # Salvaguardar cálculo bem-sucedido do interpolante.
                if C is None:
                    print("> O interpolante é None.")
                    break
                
                # Step 4. 
                C0 = rename(C, X[0])
                T = trans(X[0], X[1])
                C1 = rename(C, X[1])

                if not solver.solve([C0, T, Not(C1)]):
                    # C é invariante de T.
                    print("> O sistema é seguro.")
                    return 
                else:
                    # Step 5.1.
                    S = rename(C, X[n])
                    while True:
                        # Step 5.2.
                        T = trans(X[n], Y[m])
                        A = And(S, T)
                        if solver.solve([A, Um]):
                            print("> Não foi encontrado majorante.")
                            break 
                        else:
                            # Step 5.3.
                            C = binary_interpolant(A, Um)
                            Cn = rename(C, X[n])
                            if not solver.solve([Cn, Not(S)]):
                                # Step 5.4.
                                # C(Xn) -> S é tautologia.
                                print("> O sistema é seguro.")
                                return
                            else:
                                # Step 5.5.
                                # C(Xn) -> S não é tautologia.
                                S = Or(S, Cn)
                                
    print("> Não foi provada a segurança ou insegurança do sistema.")

model_checking(var, init, trans, error, 1, 1)

NoSolverAvailableError: Interpolator 'msat' does not support logic QF_AUFBVLIRA